In [1]:
# Some basic imports
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import warnings
import ast
import re

In [2]:
# Load the general data:
success_movies_path = 'data/processed_data/success_movies.csv'
ethnicity_path = 'data/processed_data/diversity_final.csv'

success_movies_df = pd.read_csv(success_movies_path, sep=',')
ethnicity_df = pd.read_csv(ethnicity_path, sep=',')
success_movies_df.head()

test_df = ethnicity_df.merge(success_movies_df, how='inner', on='Wikipedia_movie_ID')
test_df['Ratings'].describe()

count    21481.000000
mean         6.208342
std          1.070229
min          1.200000
25%          5.600000
50%          6.300000
75%          6.900000
max          9.300000
Name: Ratings, dtype: float64

In [3]:
test_df['Movie_box_office_revenue'].describe()

count    2.085000e+03
mean     4.665568e+07
std      1.001648e+08
min      1.020000e+04
25%      2.000000e+06
50%      9.808124e+06
75%      3.811948e+07
max      9.260471e+08
Name: Movie_box_office_revenue, dtype: float64

## Assessing Correlation between Diversity and Success

- Correlation Analysis: Evaluate the dependency between diversity and film success using: Pearson correlation (stats.pearson) for linear associations.

- Spearman correlation (stats.spearman) 

- Regression Analysis: Investigate the association type (e.g., weak, negative, positive) through linear regression.

In [4]:
# Pearson correlation coefficient use on the success variable 
stats.pearsonr(test_df['naive_diversity'],test_df['Success'])


PearsonRResult(statistic=np.float64(0.00019074658376712003), pvalue=np.float64(0.9776980804050213))

In [5]:
stats.pearsonr(test_df['naive_diversity',test_df['Movie_box_office_revenue']])

InvalidIndexError: ('naive_diversity', 0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
21476   NaN
21477   NaN
21478   NaN
21479   NaN
21480   NaN
Name: Movie_box_office_revenue, Length: 21481, dtype: float64)

KeyError: "None of [Index([ 4213160.0, 20624798.0, 24799628.0, 17841580.0, 16038391.0,  8098268.0,\n       24204980.0,  3762449.0,  2211244.0,  4441267.0,\n       ...\n          45606.0, 35484511.0, 19793556.0,  1792094.0, 33002585.0,   922145.0,\n        2856912.0,  1371014.0,  2444093.0,  1719500.0],\n      dtype='float64', length=21090)] are in the [columns]"

In [ ]:
success_movies_df.head()

0     4213160.0
1    20624798.0
2    24799628.0
3    17841580.0
4    16038391.0
Name: Wikipedia_movie_ID, dtype: float64

In [ ]:
success_movies_df.value_counts()

Wikipedia_movie_ID
1719500.0     1
4213160.0     1
9757243.0     1
3539982.0     1
21835208.0    1
             ..
24204980.0    1
8098268.0     1
16038391.0    1
17841580.0    1
24799628.0    1
Name: count, Length: 21090, dtype: int64